<a href="https://colab.research.google.com/github/namith-rive/Udemy/blob/main/ae_image_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ae_image_train

In [16]:
# -*- coding: utf-8 -*-
# 実行方法　python3 this.py Datafoler

import os
import sys
import glob
import numpy as np
import pandas as pd
import keras.models
from keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D
from keras.models import Model, Sequential
from keras.preprocessing.image import load_img, img_to_array
# tensorflow 2.9.1以降, tensorflow-gpu 2.0.0以降の場合、前行の代わりに次行を使ってください
# from tensorflow.keras.utils import load_img, img_to_array

# 入力データサイズ
INPUT_SIZE = 64
IMAGE_CHANNEL = 3
BATCH_SIZE = 2

#ModelFile = "./ae_image_model.h5"
ModelFile = '/content/drive/MyDrive/sample_image/ae_image_model.h5'

# 学習用データの読み込み
#args = sys.argv
#Datafolder = args[1]
Datafolder = '/content/drive/MyDrive/sample_image/image_train'
if not os.path.exists(Datafolder):
    print("Datafolder not exist.")
    exit()

# フォルダ配下の画像を全て読み込む
IMAGES = []
Datafiles = Datafolder + '/*.jpg'


In [17]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array

# 1枚あたり10枚の画像を水増し
N_img = 10

files = glob.glob(Datafiles)
for i, file in enumerate(files):

    img = load_img(file)
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # ImageDataGeneratorの生成
    datagen = ImageDataGenerator(
    zca_epsilon=1e-06,   # 白色化のイプシロン
    rotation_range=10.0, # ランダムに回転させる範囲
    width_shift_range=0.0, # ランダムに幅をシフトさせる範囲
    height_shift_range=0.0, # ランダムに高さをシフトさせる範囲
    brightness_range=None, # ランダムに明るさを変化させる範囲
    zoom_range=0.0,        # ランダムにズームさせる範囲
    horizontal_flip=True, # ランダムに水平方向に反転させる
    vertical_flip=True, # ランダムに垂直方向に反転させる
    )

    # 画像を水増し生成
    dg = datagen.flow(x, batch_size=1, save_to_dir=Datafolder, save_prefix='img', save_format='jpg')
    for i in range(N_img):
        batch = dg.next()

In [18]:

for pic in glob.glob(Datafiles):
    img = img_to_array(load_img(pic, target_size=(INPUT_SIZE, INPUT_SIZE, IMAGE_CHANNEL)))
    IMAGES.append(img)
# arrayに変換
IMAGES = np.array(IMAGES)
# 画素値を0から1の範囲に変換
IMAGES = IMAGES.astype('float32')
train_data = IMAGES / 255.0
print('Number of Images: ', len(train_data))

# AutoEncoder + Convolution モデルの組み立て
model = Sequential()

# 3*3フィルタ32枚をゼロパディングで埋めて畳み込む
model.add(Conv2D(int(INPUT_SIZE/2), (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Conv2D(int(INPUT_SIZE/4), (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Conv2D(int(INPUT_SIZE/4), (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(UpSampling2D((2, 2)))

model.add(Conv2D(int(INPUT_SIZE/2), (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(UpSampling2D((2, 2)))

model.add(Conv2D(IMAGE_CHANNEL, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.build((BATCH_SIZE, INPUT_SIZE, INPUT_SIZE, IMAGE_CHANNEL))
model.summary()

# 学習：入力データと結果データを同一にする
history = model.fit(x=train_data, y=train_data,
                             epochs=100,
                             batch_size=BATCH_SIZE,
                             validation_split=0.1)


# 学習済みモデルを保存
model.save(ModelFile)
print('Model saved.')



Number of Images:  347
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (2, 64, 64, 32)           896       
                                                                 
 batch_normalization_5 (Bat  (2, 64, 64, 32)           128       
 chNormalization)                                                
                                                                 
 activation_5 (Activation)   (2, 64, 64, 32)           0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (2, 32, 32, 32)           0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (2, 32, 32, 16)           4624      
                                                                 
 batch_normalization_6 (Bat  (2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# -*- coding: utf-8 -*-
# 実行方法　python3 this.py datafile

import os
import sys
import numpy as np
import keras.models
from keras.preprocessing.image import load_img, img_to_array
# tensorflow 2.9.1以降, tensorflow-gpu 2.0.0以降の場合、前行の代わりに次行を使ってください
# from tensorflow.keras.utils import load_img, img_to_array

# 入力データサイズ
INPUT_SIZE = 64
IMAGE_CHANNEL = 3

#ModelFile = "./ae_image_model.h5"

if not os.path.exists(ModelFile):
    print("Modelfile not exist.")
    exit()

#args = sys.argv
#Datafile = args[1]
Datafile = '/content/drive/MyDrive/sample_image/image_detect/bolt_normal.jpg'
Datafile2 = '/content/drive/MyDrive/sample_image/image_detect/bolt_anomaly.jpg'

if not os.path.exists(Datafile):
    print("Datafile not exist.")
    exit()

model = keras.models.load_model(ModelFile)

#正常画像
img = img_to_array(load_img(Datafile, target_size=(INPUT_SIZE, INPUT_SIZE, IMAGE_CHANNEL)))
# arrayに変換
img = np.array(img)
img = img.reshape(1, INPUT_SIZE, INPUT_SIZE, IMAGE_CHANNEL)
# 画素値を0から1の範囲に変換
img = img.astype('float32')
detect_data = img / 255.0

detect_pred = model.predict(detect_data)

detect_score = np.mean(np.square(detect_data - detect_pred), axis=1)
detect_score = np.average(detect_score)
print('Score: ', detect_score)

# 異常画像
img = img_to_array(load_img(Datafile2, target_size=(INPUT_SIZE, INPUT_SIZE, IMAGE_CHANNEL)))
# arrayに変換
img = np.array(img)
img = img.reshape(1, INPUT_SIZE, INPUT_SIZE, IMAGE_CHANNEL)
# 画素値を0から1の範囲に変換
img = img.astype('float32')
detect_data = img / 255.0

detect_pred = model.predict(detect_data)

detect_score = np.mean(np.square(detect_data - detect_pred), axis=1)
detect_score = np.average(detect_score)
print('Score: ', detect_score)

1/1 [==============================] - 0s 128ms/step
Score:  0.0023131517
1/1 [==============================] - 0s 19ms/step
Score:  0.0041914363


1/1 [==============================] - 0s 111ms/step

Score:  0.0033047695

1/1 [==============================] - 0s 19ms/step

Score:  0.010835145

In [13]:
IMAGES.shape

(32, 64, 64, 3)

In [12]:
img.shape

(1, 64, 64, 3)

In [11]:
img[0]

array([[[165., 166., 158.],
        [167., 167., 157.],
        [172., 170., 158.],
        ...,
        [165., 162., 153.],
        [158., 158., 150.],
        [162., 163., 157.]],

       [[165., 166., 158.],
        [163., 163., 153.],
        [165., 163., 151.],
        ...,
        [165., 162., 153.],
        [158., 159., 151.],
        [157., 160., 153.]],

       [[168., 169., 161.],
        [168., 168., 158.],
        [168., 168., 158.],
        ...,
        [166., 164., 152.],
        [166., 163., 154.],
        [160., 160., 150.]],

       ...,

       [[163., 163., 153.],
        [164., 165., 149.],
        [163., 164., 148.],
        ...,
        [166., 164., 149.],
        [167., 165., 150.],
        [165., 163., 148.]],

       [[167., 167., 157.],
        [168., 169., 153.],
        [168., 169., 153.],
        ...,
        [163., 162., 144.],
        [165., 164., 146.],
        [164., 163., 145.]],

       [[161., 161., 151.],
        [161., 162., 146.],
        [165., 1